# 简介

> 指导文章：[14. PEFT：在大模型中快速应用 LoRA](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/14.%20PEFT：在大模型中快速应用%20LoRA.md)

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/12-lora-peft) | [Colab](https://colab.research.google.com/drive/1-gWfn9xslSq6WlYDS9cinnyDEhBhjte4?usp=sharing)


## 安装必要的库

In [1]:
!uv add transformers
!uv add peft

Resolved 220 packages in 1ms
Audited 205 packages in 4ms
Resolved 220 packages in 1ms
Audited 205 packages in 4ms


## 加载预训练模型

In [2]:
import os
# 设置模型下载镜像
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载预训练的 GPT-2 模型和分词器
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

# 使用eos_token作为pad_token
tokenizer.pad_token = tokenizer.eos_token

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## 使用 PEFT 应用 LoRA

In [3]:
from peft import get_peft_model, LoraConfig, TaskType

# 配置 LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 任务类型：因果语言模型
    inference_mode=False,          # 推理模式关闭，以进行训练
    r=8,                           # 低秩值 r
    lora_alpha=32,                 # LoRA 的缩放因子
    lora_dropout=0.1,              # Dropout 概率
)

# 将 LoRA 应用到模型中
model = get_peft_model(model, lora_config)

/root/AI/AI-Guide-and-Demos-zh_CN/.venv/lib/python3.12/site-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## 查看当前模型架构

In [4]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
           

## 查看增加的参数量

应用 LoRA 后，我们一般都希望了解模型参数量的变化，它的计算其实很简单。

### 理论计算

对于每个应用了 LoRA 的层，增加的参数量为：

$$
\text{增加的参数量} = r \times (\text{输入维度} + \text{输出维度})
$$

- **`r`**：LoRA 的低秩值。
- **输入维度**：层的输入特征数。
- **输出维度**：层的输出特征数。

### 使用 PEFT 查看参数

`peft` 提供了查看模型参数的便捷方法：

In [5]:
# 查看 LoRA 模块
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


In [6]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        all_params += num_params
        if param.requires_grad:
            trainable_params += num_params
    print(f"可训练参数量: {trainable_params}")
    print(f"总参数量: {all_params}")
    print(f"可训练参数占比: {100 * trainable_params / all_params:.2f}%")
    
print_trainable_parameters(model)

可训练参数量: 294912
总参数量: 124734720
可训练参数占比: 0.24%


## 准备数据

下面使用公开数据集进行演示。

In [7]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

# 1. 使用英文小说数据集
# dataset = load_dataset("roneneldan/TinyStories", split="train[:1000]")  # 取前1000条

# 2. 使用 imdb 电影评论数据集
dataset = load_dataset("imdb", split="train[:500]")

print(f"数据集大小: {len(dataset)}")
print(f"数据集列名: {dataset.column_names}")
print(f"示例数据: {dataset[0]}")

def preprocess_function(examples):
    """数据预处理函数"""
    # 对于IMDB数据集，我们使用 'text' 字段
    # 你可以根据不同数据集调整字段名
    texts = examples['text']
    
    # Tokenization
    model_inputs = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512,  # 根据需要调整最大长度
        return_tensors="pt" if len(texts) == 1 else None
    )
    
    # 对于因果语言模型，labels就是input_ids
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    
    return model_inputs

# 应用预处理
print("正在预处理数据...")
train_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names,  # 移除原始列，只保留模型需要的
    desc="Tokenizing dataset"
)

print(f"预处理后的数据集大小: {len(train_dataset)}")
print(f"预处理后的数据集特征: {train_dataset.features}")

# 创建数据整理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 不使用masked language modeling，因为我们做的是causal LM
    pad_to_multiple_of=8  # 为了提高效率，填充到8的倍数
)

数据集大小: 500
数据集列名: ['text', 'label']
示例数据: {'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, th

## 开始微调

In [8]:
from transformers import Trainer, TrainingArguments

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',         # 模型保存和日志输出的目录路径
    num_train_epochs=3,             # 训练的总轮数（epochs）
    per_device_train_batch_size=4,  # 每个设备（如GPU或CPU）上的训练批次大小，4表示每次输入模型的数据数量
    learning_rate=5e-5,             # 学习率
    logging_steps=10,               # 每隔多少步（steps）进行一次日志记录
    save_steps=100,                 # 每隔多少步保存模型
    report_to="none",               # 不使用wandb等工具记录
)

# 创建 Trainer
trainer = Trainer(
    model=model,                    # 训练的模型对象，需要事先加载好
    args=training_args,             # 上面定义的训练参数配置
    train_dataset=train_dataset,    # 使用预处理后的数据集
    data_collator=data_collator     # 数据整理器
)

# 开始训练
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.012300
20,3.886300
30,3.798400
40,3.836100
50,3.810400
60,3.854100
70,3.807300
80,3.875300
90,3.780300
100,3.771700


TrainOutput(global_step=375, training_loss=3.813963150024414, metrics={'train_runtime': 26.4838, 'train_samples_per_second': 56.638, 'train_steps_per_second': 14.16, 'total_flos': 393297002496000.0, 'train_loss': 3.813963150024414, 'epoch': 3.0})

### 保存和加载 LoRA 微调的模型

训练完成后，你可以保存或者加载 LoRA 微调的参数，下面是个简单的示例。

In [9]:
# 保存 LoRA 参数
model.save_pretrained('./lora_model')

In [10]:
# 加载原始模型
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# 加载 LoRA 参数
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, './lora_model')

### 合并 LoRA 权重并卸载 PEFT 包装

在完成微调后，可以使用 `merge_and_unload()` 将 LoRA 的权重合并回原始模型。这在部署和推理阶段非常有用，因为这样可以：

- **减少依赖**：合并后，模型成为标准的 `transformers` 模型，不再需要 `peft` 库。
- **提高推理效率**：减少了额外的计算开销，推理速度可能会有所提升。
- **简化模型保存和加载**：不需要分别保存基础模型和 LoRA 参数。

运行下面的代码：

In [11]:
# 对比合并前后的模型
print("合并前的模型结构：")
print(model)

# 合并并卸载 LoRA 权重
model = model.merge_and_unload()

print("合并后的模型结构：")
print(model)

合并前的模型结构：
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
 

你应该注意到，合并后模型的 LoRA 层将被去除。

现在，你可以像保存普通模型一样保存：

In [12]:
# 保存合并后的模型
model.save_pretrained('./merged_model')
tokenizer.save_pretrained('./merged_model')

('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/vocab.json',
 './merged_model/merges.txt',
 './merged_model/added_tokens.json',
 './merged_model/tokenizer.json')

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载合并后的模型
tokenizer = AutoTokenizer.from_pretrained('./merged_model')
model = AutoModelForCausalLM.from_pretrained('./merged_model')

# 进行推理
inputs = tokenizer("Hello, World！", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, World！！！！！！！�


**注意**：

- **不可逆操作**：合并操作是不可逆的。如果你之后还需要进一步微调 LoRA 参数，需确保在合并前备份模型。
- **无需 PEFT 库**：合并后的模型不再包含 LoRA 适配器（Adapter）的信息，因此在加载时无需使用 `PeftModel`。

## 可能的错误及解决方案（TypeError: Expected state_dict to be dict-like...）

在使用 `PEFT` 和 `LoRA` 进行模型微调和保存加载时，可能会遇到如下错误：

```
TypeError: Expected state_dict to be dict-like, got <class 'peft.peft_model.PeftModel'>.
```

### 错误原因

一般是因为混合使用不同的保存和加载方式，这个错误不局限于 `PeftModel`，问题出在你用 `torch.save(model)` 保存整个模型却用 `load_state_dict()` 去加载，注意模型加载和保存的一致性。

### 错误重现

下面我们来复现它，看是不是和你的操作一致（这里以 `PeftModel` 举例）：

#### 1. 错误地保存整个 `PeftModel` 对象而不是其 `state_dict`：

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# 加载预训练模型和分词器
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

# 配置 LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# 应用 LoRA
model = get_peft_model(model, lora_config)

# 错误地保存整个 PeftModel 对象
torch.save(model, './model')

/root/AI/AI-Guide-and-Demos-zh_CN/.venv/lib/python3.12/site-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


#### 2. 加载时传入 `PeftModel` 对象

In [15]:
# 初始化模型
model = AutoModelForCausalLM.from_pretrained("gpt2")

# 错误地加载模型，期望接收 state_dict 但实际加载了整个模型对象
model.load_state_dict(torch.load('./model'))  # 这里会报错

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL peft.peft_model.PeftModelForCausalLM was not an allowed global by default. Please use `torch.serialization.add_safe_globals([peft.peft_model.PeftModelForCausalLM])` or the `torch.serialization.safe_globals([peft.peft_model.PeftModelForCausalLM])` context manager to allowlist this global if you trust this class/function.

Check the documentation of

### 解决方法

确保你保存和加载的对象是一致的：

- `torch.save(model, '...')` 对应于 `torch.load(model, '...')`。
- `torch.save(model.state_dict(), '...')` 对应于 `model.load_state_dict(torch.load('...'))`

## 一个非常刁钻的错误：使用 get_peft_model() 后再导入 LoRA

In [16]:
import torch
import torch.nn as nn
from torch.optim import Adam
from copy import deepcopy
from peft import get_peft_model, LoraConfig, PeftModel

# 固定随机数种子，确保结果可复现
torch.manual_seed(42)

# 定义一个简单的线性模型
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# 实例化线性模型
model = LinearModel(input_size=10, output_size=1)

# 在应用 LoRA 之前深拷贝原始模型，确保后续公平比较
original_model = deepcopy(model)

# 配置 LoRA 参数
config = LoraConfig(
    inference_mode=False,
    r=4,
    lora_alpha=16,
    target_modules=['linear'],
)

# 将 LoRA 应用到模型中
lora_model = get_peft_model(model, config)

# 定义一个简单的损失函数和优化器
criterion = nn.MSELoss()
optimizer = Adam(lora_model.parameters(), lr=1e-3)

# 生成一些模拟的训练数据
input_data = torch.randn(100, 10)  # 100 个样本，每个样本有 10 个特征
target_data = torch.randn(100, 1)  # 对应的目标值

# 训练一个回合
lora_model.train()
for epoch in range(1):  # 训练 1 个回合
    optimizer.zero_grad()
    outputs = lora_model(input_data)
    loss = criterion(outputs, target_data)
    loss.backward()
    optimizer.step()

# 训练后保存 LoRA 权重
lora_model.save_pretrained('linear_lora_model')

# 方法 1：先使用 get_peft_model，再加载 LoRA 权重
model1 = PeftModel.from_pretrained(get_peft_model(deepcopy(original_model), config), 'linear_lora_model')

# 方法 2：直接加载 LoRA 权重
model2 = PeftModel.from_pretrained(deepcopy(original_model), 'linear_lora_model')

# 生成相同的输入数据以进行输出比较
test_input = torch.randn(1, 10)

# 比较四个模型的输出（原始模型，LoRA，方法1，方法2）
def compare_model_outputs(input_data):
    # 原始模型
    original_output = original_model(input_data)
    print("原始模型输出:", original_output.detach().numpy())

    # 训练后的 LoRA 模型
    lora_output = lora_model(input_data)
    print("训练后的 LoRA 模型输出:", lora_output.detach().numpy())

    # 方法 1：先使用 get_peft_model，再加载 LoRA
    output1 = model1(input_data)
    print("方法 1（先使用 get_peft_model，再加载 LoRA）输出:", output1.detach().numpy())

    # 方法 2：直接加载 LoRA
    output2 = model2(input_data)
    print("方法 2（直接加载 LoRA）输出:", output2.detach().numpy())

    if torch.allclose(original_output, output1):
        print("\n原始模型和方法 1 输出相同。")
    if torch.allclose(lora_output, output2):
        print("训练后的 LoRA 模型和方法 2 输出相同。\n")

# 比较两个模型的参数
def compare_params(m1, m2):
    for (n1, p1), (n2, p2) in zip(m1.named_parameters(), m2.named_parameters()):
        if n1 != n2 or not torch.allclose(p1, p2):
            print(f"参数不匹配: \n{n1}\n{n2}")
            return False
    return True

# 比较四个模型的输出
compare_model_outputs(test_input)

# 检查方法 1 和方法 2 的参数是否一致
if compare_params(model1, model2):
    print("方法 1 和方法 2 的 LoRA 模型参数一致！")
else:
    print("方法 1 和方法 2 的 LoRA 模型参数不一致！")

原始模型输出: [[-0.03600371]]
训练后的 LoRA 模型输出: [[-0.03428639]]
方法 1（先使用 get_peft_model，再加载 LoRA）输出: [[-0.03600371]]
方法 2（直接加载 LoRA）输出: [[-0.03428639]]

原始模型和方法 1 输出相同。
训练后的 LoRA 模型和方法 2 输出相同。

参数不匹配: 
base_model.model.base_model.model.linear.base_layer.weight
base_model.model.linear.base_layer.weight
方法 1 和方法 2 的 LoRA 模型参数不一致！


/root/AI/AI-Guide-and-Demos-zh_CN/.venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/root/AI/AI-Guide-and-Demos-zh_CN/.venv/lib/python3.12/site-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.linear.lora_A.default.weight', 'base_model.model.base_model.model.linear.lora_B.default.weight'].
  warnings.warn(warn_message)
